In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests6

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)
#NOTE! Weird size distortions with shi's test when theta = .5....

(1000, 1) (1, 1)


# Ex 1

In [3]:
beta = 1

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.88 & 1.00 & 0.85 & 0.83 & 0.99 & 0.94   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.12 & 0.00 & 0.15 & 0.17 & 0.01 & 0.06   \\
\hline
\end{tabular}
(array([0.88, 0.  , 0.12]), array([1., 0., 0.]), array([0.85, 0.  , 0.15]), array([0.83, 0.  , 0.17]), array([0.99, 0.  , 0.01]), array([0.94, 0.  , 0.06]), 14.377140600040564, 53.88525806846468, 51.962043821100764)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.99 & 0.91 & 0.90 & 0.97 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.07 & 0.00 & 0.08 & 0.10 & 0.03 & 0.03   \\
\hline
\end{tabular}
(array([0.92, 0.01, 0.07]), array([0.99, 0.01, 0.  ]), array([0.91, 0.01, 0.08]), array([0.9, 0. , 0.1]), array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), 10.140459522966092, 87.00853854089654, 79.28891148345271)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.92 & 0.92 & 0.92 & 0.93 & 0.99   \\
Model 1 & 0.02 & 0.02 & 0.03 & 0.02 & 0.01 & 0.00   \\
Model 2 & 0.06 & 0.06 & 0.05 & 0.06 & 0.06 & 0.01   \\
\hline
\end{tabular}
(array([0.92, 0.02, 0.06]), array([0.92, 0.02, 0.06]), array([0.92, 0.03, 0.05]), array([0.92, 0.02, 0.06]), array([0.93, 0.01, 0.06]), array([0.99, 0.  , 0.01]), 17.959732835605983, 114.33942068301658, 115.5700023970471)


In [7]:
beta = 1.5

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.99 & 0.88 & 0.86 & 0.97 & 0.97   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.09 & 0.00 & 0.10 & 0.13 & 0.03 & 0.03   \\
\hline
\end{tabular}
(array([0.9 , 0.01, 0.09]), array([0.99, 0.01, 0.  ]), array([0.88, 0.02, 0.1 ]), array([0.86, 0.01, 0.13]), array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), 9.830318412412273, 87.32110795502852, 75.53928052746704)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.94 & 0.94 & 0.94 & 0.96 & 0.99   \\
Model 1 & 0.03 & 0.03 & 0.02 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.04 & 0.03 & 0.04 & 0.04 & 0.02 & 0.01   \\
\hline
\end{tabular}
(array([0.93, 0.03, 0.04]), array([0.94, 0.03, 0.03]), array([0.94, 0.02, 0.04]), array([0.94, 0.02, 0.04]), array([0.96, 0.02, 0.02]), array([0.99, 0.  , 0.01]), 3.9603375692475624, 116.86107155510963, 119.92977962156846)


In [24]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.94 & 0.95 & 0.95 & 0.95 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.05 & 0.05 & 0.05 & 0.05 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([0.95, 0.  , 0.05]), array([0.95, 0.  , 0.05]), array([0.95, 0.  , 0.05]), array([1., 0., 0.]), 25.061344875636113, 103.42032548151221, 117.94595366109849)


# Power

In [11]:
def gen_data2(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([0,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

In [12]:
beta = 1.

In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.48 & 1.00 & 0.51 & 0.35 & 0.94 & 0.74   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.52 & 0.00 & 0.49 & 0.65 & 0.06 & 0.26   \\
\hline
\end{tabular}
(array([0.48, 0.  , 0.52]), array([1., 0., 0.]), array([0.51, 0.  , 0.49]), array([0.35, 0.  , 0.65]), array([0.94, 0.  , 0.06]), array([0.74, 0.  , 0.26]), -35.13631953828803, 21.487395303375628, 24.755168830820693)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.11 & 1.00 & 0.10 & 0.09 & 0.27 & 0.31   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.89 & 0.00 & 0.90 & 0.91 & 0.73 & 0.69   \\
\hline
\end{tabular}
(array([0.11, 0.  , 0.89]), array([1., 0., 0.]), array([0.1, 0. , 0.9]), array([0.09, 0.  , 0.91]), array([0.27, 0.  , 0.73]), array([0.31, 0.  , 0.69]), -111.78201166542338, 27.18659153133561, 26.864415668183074)


In [15]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.01 & 0.92 & 0.01 & 0.00 & 0.01 & 0.08   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.99 & 0.08 & 0.99 & 1.00 & 0.99 & 0.92   \\
\hline
\end{tabular}
(array([0.01, 0.  , 0.99]), array([0.92, 0.  , 0.08]), array([0.01, 0.  , 0.99]), array([0., 0., 1.]), array([0.01, 0.  , 0.99]), array([0.08, 0.  , 0.92]), -238.45200313196827, 30.48578279085797, 31.62716451235375)


In [16]:
beta = 1.5

In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.11 & 0.99 & 0.10 & 0.06 & 0.40 & 0.46   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.89 & 0.01 & 0.90 & 0.94 & 0.60 & 0.54   \\
\hline
\end{tabular}
(array([0.11, 0.  , 0.89]), array([0.99, 0.  , 0.01]), array([0.1, 0. , 0.9]), array([0.06, 0.  , 0.94]), array([0.4, 0. , 0.6]), array([0.46, 0.  , 0.54]), -100.63356852233046, 20.697015775983946, 26.69892744159029)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.02 & 0.93 & 0.01 & 0.01 & 0.03 & 0.10   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.98 & 0.07 & 0.99 & 0.99 & 0.97 & 0.90   \\
\hline
\end{tabular}
(array([0.02, 0.  , 0.98]), array([0.93, 0.  , 0.07]), array([0.01, 0.  , 0.99]), array([0.01, 0.  , 0.99]), array([0.03, 0.  , 0.97]), array([0.1, 0. , 0.9]), -270.9075752887447, 32.2895572361591, 33.940698421240775)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, beta =beta)
mc_out = vuong_tests6.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.00 & 0.82 & 0.00 & 0.00 & 0.00 & 0.02   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 1.00 & 0.18 & 1.00 & 1.00 & 1.00 & 0.98   \\
\hline
\end{tabular}
(array([0., 0., 1.]), array([0.82, 0.  , 0.18]), array([0., 0., 1.]), array([0., 0., 1.]), array([0., 0., 1.]), array([0.02, 0.  , 0.98]), -548.5951814929487, 36.953103812309266, 40.844329246285945)
